<a href="https://colab.research.google.com/github/schettinina/MONAN---Back-end/blob/main/Barreiras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ollama pandas requests PyPDF2

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
import time
subprocess.Popen(['ollama', 'serve'])
time.sleep(5)  # Aguardar inicialização

In [ ]:
!ollama pull deepseek-coder-v2

In [ ]:
import subprocess
import requests
from io import BytesIO
import PyPDF2
import time

def baixar_pdf_github(url):
    """Baixa PDF do GitHub"""
    try:
        if 'github.com' in url and '/blob/' in url:
            url = url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')

        print(f"📥 Baixando PDF de: {url}")
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        return BytesIO(response.content)
    except Exception as e:
        print(f"❌ Erro ao baixar: {str(e)}")
        return None

def extrair_texto_pdf(pdf_stream):
    """Extrai texto do PDF"""
    try:
        leitor = PyPDF2.PdfReader(pdf_stream)
        texto = ""

        for pagina in leitor.pages:
            texto_pagina = pagina.extract_text()
            if texto_pagina:
                texto += texto_pagina + "\n"

        print(f"✅ Texto extraído: {len(texto)} caracteres")
        return texto
    except Exception as e:
        print(f"❌ Erro ao extrair texto: {str(e)}")
        return ""

def consultar_deepseek(prompt):
    """Consulta o DeepSeek Coder V2 via Ollama"""
    try:
        # Limitar tamanho do prompt para evitar erros
        if len(prompt) > 15000:
            prompt = prompt[:15000] + "... [texto truncado por limite]"

        comando = f'ollama run deepseek-coder-v2 "{prompt}"'

        print("🧠 Consultando DeepSeek Coder V2...")
        resultado = subprocess.run(
            comando,
            shell=True,
            capture_output=True,
            text=True,
            timeout=180,
            encoding='utf-8',
            errors='ignore'
        )

        if resultado.returncode == 0:
            resposta = resultado.stdout.strip()

            # Limpar resposta (remover prompts repetidos)
            linhas = resposta.split('\n')
            linhas_validas = [l for l in linhas if not l.startswith('>>>') and not l.startswith('```')]
            resposta_limpa = '\n'.join(linhas_validas)

            return resposta_limpa
        else:
            return f"Erro: {resultado.stderr}"

    except subprocess.TimeoutExpired:
        return "⏱️ Tempo limite excedido (3 minutos)"
    except Exception as e:
        return f"Erro na consulta: {str(e)}"

def criar_prompt_principal(texto_pdf):
    """Cria o prompt principal para a pergunta específica"""

    # Limitar o contexto para focar nas partes mais relevantes
    contexto = texto_pdf[:12000] if len(texto_pdf) > 12000 else texto_pdf

    # Encontrar seções relevantes
    secoes_relevantes = []
    palavras_chave = ["mulher", "mulheres", "feminino", "gênero", "acesso", "barreira", "SUS", "saúde"]

    linhas = texto_pdf.split('\n')
    for i, linha in enumerate(linhas):
        if any(palavra.lower() in linha.lower() for palavra in palavras_chave):
            # Pegar contexto ao redor
            inicio = max(0, i - 2)
            fim = min(len(linhas), i + 3)
            contexto_local = '\n'.join(linhas[inicio:fim])
            secoes_relevantes.append(contexto_local)

    # Combinar seções relevantes se encontradas
    if secoes_relevantes:
        contexto = "\n---\n".join(secoes_relevantes[:10])  # Limitar a 10 seções
        print(f"✅ Encontradas {len(secoes_relevantes)} seções relevantes")

    prompt = f"""
## TAREFA PRINCIPAL:
Analise o artigo científico fornecido e responda COMPLETAMENTE à seguinte questão:

**"Quais as principais barreiras de acesso das mulheres aos serviços de saúde no contexto do Sistema Único de Saúde?"**

## CONTEXTO DO ARTIGO (trechos relevantes):
{contexto}

## INSTRUÇÕES DETALHADAS:

1. **FOCO NA PERGUNTA**: Responda EXCLUSIVAMENTE sobre barreiras de acesso no SUS

2. **ESTRUTURA DA RESPOSTA**:
   a) INTRODUÇÃO: Contextualize brevemente (1-2 parágrafos)
   b) BARREIRAS PRINCIPAIS: Liste e explique cada uma (mínimo 5, máximo 10)
   c) EVIDÊNCIAS DO ARTIGO: Cite trechos/dados específicos do artigo
   d) INTERSECCIONALIDADE: Como raça, classe, região afetam o acesso
   e) IMPLICAÇÕES: O que isso significa para políticas públicas
   f) CONCLUSÃO: Síntese final

3. **FORMATAÇÃO**:
   - Use tópicos claros
   - Seja específico com exemplos do artigo
   - Use linguagem acadêmica mas acessível
   - Cite páginas ou seções quando possível
   - Destaque dados quantitativos se presentes

4. **SE O ARTIGO NÃO ABORDAR SUFICIENTEMENTE**:
   - Indique claramente as limitações
   - Destaque o que ELE aborda
   - Sugira que a resposta é baseada apenas no conteúdo disponível

5. **CRITÉRIOS PARA IDENTIFICAR BARREIRAS**:
   - Geográficas/territoriais
   - Econômicas/financeiras
   - Organizacionais/administrativas
   - Culturais/sociais
   - Informacionais
   - Específicas por ciclo de vida

## EXEMPLO DE ESTRUTURA IDEAL:

**INTRODUÇÃO**
[Contexto sobre equidade de gênero no SUS]

**PRINCIPAIS BARREIRAS IDENTIFICADAS**

1. **Barreira Geográfica**
   - Descrição: [o que o artigo diz]
   - Evidência: [trecho específico]
   - Impacto: [consequências]

2. **Barreira Econômica**
   - Descrição: [o que o artigo diz]
   - Evidência: [trecho específico]
   - Impacto: [consequências]

[Continue com outras barreiras...]

**INTERSECCIONALIDADE**
[Como fatores se combinam]

**RECOMENDAÇÕES IMPLÍCITAS**
[O que o artigo sugere]

**CONCLUSÃO**
[Síntese final baseada no artigo]

## RESPOSTA FINAL (em português brasileiro):
"""
    return prompt

def criar_prompt_verificacao(texto_pdf, resposta_principal):
    """Cria prompt para verificar consistência da resposta"""

    contexto = texto_pdf[:8000] if len(texto_pdf) > 8000 else texto_pdf

    prompt = f"""
## TAREFA DE VERIFICAÇÃO:
Verifique se a resposta abaixo está CORRETAMENTE baseada no artigo fornecido.

## ARTIGO ORIGINAL (trecho):
{contexto}

## RESPOSTA PARA VERIFICAR:
{resposta_principal[:2000]}

## PERGUNTAS PARA VERIFICAÇÃO:
1. A resposta cita informações que realmente estão no artigo?
2. Alguma barreira foi "inventada" ou exagerada?
3. Foram omitidas barreiras importantes que o artigo menciona?
4. A análise de interseccionalidade é apoiada pelo artigo?

## INSTRUÇÕES:
- Liste APENAS erros de interpretação ou omissões graves
- Se a resposta for fiel, escreva "RESPOSTA FIEL AO ARTIGO"
- Seja específico: "O artigo não menciona X, mas a resposta diz que menciona"
- Limite a 3-5 pontos principais

## VERIFICAÇÃO:
"""
    return prompt

def salvar_resultados(resposta, url, verificacao=""):
    """Salva os resultados em arquivos"""

    timestamp = time.strftime("%Y%m%d_%H%M%S")

    # Arquivo principal
    with open(f"resposta_barreiras_sus_{timestamp}.txt", "w", encoding="utf-8") as f:
        f.write("="*80 + "\n")
        f.write("RESPOSTA: Barreiras de acesso das mulheres ao SUS\n")
        f.write("="*80 + "\n\n")
        f.write(f"Artigo analisado: {url}\n")
        f.write(f"Data/hora: {time.strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write("="*80 + "\n\n")
        f.write("PERGUNTA:\n")
        f.write('"Quais as principais barreiras de acesso das mulheres aos serviços\n')
        f.write('de saúde no contexto do Sistema Único de Saúde?"\n\n')
        f.write("="*80 + "\n\n")
        f.write("RESPOSTA BASEADA NO ARTIGO:\n\n")
        f.write(resposta)

        if verificacao:
            f.write("\n" + "="*80 + "\n")
            f.write("VERIFICAÇÃO DE CONSISTÊNCIA:\n\n")
            f.write(verificacao)

    # Resumo executivo
    with open(f"resumo_barreiras_{timestamp}.txt", "w", encoding="utf-8") as f:
        f.write("RESUMO EXECUTIVO - Barreiras das mulheres ao SUS\n")
        f.write("="*60 + "\n\n")

        # Extrair principais pontos da resposta
        linhas = resposta.split('\n')
        barreiras = []

        for linha in linhas:
            linha_limpa = linha.strip()
            if (linha_limpa.startswith('**') and 'barreira' in linha_limpa.lower()) or \
               (linha_limpa.startswith('-') and 'barreira' in linha_limpa.lower()) or \
               (linha_limpa.startswith('1.') or linha_limpa.startswith('2.') or
                linha_limpa.startswith('3.') or linha_limpa.startswith('4.') or
                linha_limpa.startswith('5.')):
                barreiras.append(linha_limpa)

        if barreiras:
            f.write("PRINCIPAIS BARREIRAS IDENTIFICADAS:\n")
            for barreira in barreiras[:10]:  # Limitar a 10
                f.write(f"• {barreira}\n")
        else:
            f.write("Principais achados do artigo:\n")
            # Pegar primeiras 20 linhas da resposta
            for linha in linhas[:20]:
                if len(linha.strip()) > 10:
                    f.write(f"{linha}\n")

    print(f"\n💾 Resultados salvos:")
    print(f"   • resposta_barreiras_sus_{timestamp}.txt")
    print(f"   • resumo_barreiras_{timestamp}.txt")

def main():
    print("""
    ============================================
    🎯 ANALISADOR DE BARREIRAS - MULHERES NO SUS
    ============================================

    Este código analisará um artigo científico e responderá:

    "Quais as principais barreiras de acesso das mulheres
    aos serviços de saúde no contexto do Sistema Único de Saúde?"

    ============================================
    """)

    # Verificar Ollama
    try:
        print("🔍 Verificando Ollama...")
        subprocess.run(["ollama", "--version"], capture_output=True, check=True)

        # Verificar modelo
        print("🔍 Verificando modelo deepseek-coder-v2...")
        modelos = subprocess.run(["ollama", "list"], capture_output=True, text=True)
        if "deepseek-coder-v2" not in modelos.stdout:
            print("📥 Modelo não encontrado. Baixando deepseek-coder-v2...")
            subprocess.run(["ollama", "pull", "deepseek-coder-v2"], check=True)
    except Exception as e:
        print(f"❌ Erro no Ollama: {str(e)}")
        print("📥 Instale em: https://ollama.ai/")
        print("📥 Depois: ollama pull deepseek-coder-v2")
        return

    # Solicitar URL
    print("\n" + "="*60)
    url = input("🔗 Cole a URL do PDF no GitHub: ").strip()

    if not url:
        print("❌ URL vazia. Encerrando.")
        return

    # Baixar e processar PDF
    print("\n" + "="*60)
    pdf_stream = baixar_pdf_github(url)

    if not pdf_stream:
        print("❌ Falha ao baixar o PDF.")
        return

    texto = extrair_texto_pdf(pdf_stream)

    if not texto or len(texto) < 100:
        print("❌ Texto insuficiente extraído do PDF.")
        return

    # Verificar conteúdo relevante
    termos = ["mulher", "mulheres", "saúde", "SUS", "acesso", "gênero"]
    encontrados = [t for t in termos if t.lower() in texto.lower()]

    if len(encontrados) < 2:
        print(f"⚠️  Poucos termos relevantes encontrados: {encontrados}")
        print("   O artigo pode não focar em saúde da mulher no SUS")
        resposta = input("   Continuar mesmo assim? (s/n): ").lower()
        if resposta != 's':
            return

    # Gerar resposta principal
    print("\n" + "="*60)
    print("🧠 Analisando artigo e gerando resposta...")
    print("⏳ Isso pode levar 2-5 minutos...")

    prompt_principal = criar_prompt_principal(texto)
    resposta_principal = consultar_deepseek(prompt_principal)

    if not resposta_principal or len(resposta_principal) < 100:
        print("❌ Resposta muito curta ou vazia.")
        return

    # Verificação opcional
    print("\n" + "="*60)
    print("🔍 Verificando consistência da resposta...")

    prompt_verificacao = criar_prompt_verificacao(texto, resposta_principal)
    verificacao = consultar_deepseek(prompt_verificacao)

    # Exibir resultados
    print("\n" + "="*80)
    print("🎯 RESPOSTA FINAL")
    print("="*80)
    print(resposta_principal)

    if verificacao and "RESPOSTA FIEL" not in verificacao.upper():
        print("\n" + "="*80)
        print("⚠️  VERIFICAÇÃO DE CONSISTÊNCIA")
        print("="*80)
        print(verificacao)

    # Salvar resultados
    salvar_resultados(resposta_principal, url, verificacao)

    print("\n" + "="*80)
    print("✅ ANÁLISE CONCLUÍDA!")
    print("="*80)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n❌ Interrompido pelo usuário.")
    except Exception as e:
        print(f"\n❌ Erro inesperado: {str(e)}")


    🎯 ANALISADOR DE BARREIRAS - MULHERES NO SUS
    
    Este código analisará um artigo científico e responderá:
    
    "Quais as principais barreiras de acesso das mulheres 
    aos serviços de saúde no contexto do Sistema Único de Saúde?"
    
    
🔍 Verificando Ollama...
🔍 Verificando modelo deepseek-coder-v2...

🔗 Cole a URL do PDF no GitHub:   https://github.com/biel1205/testenovembro/raw/main/Akter2018_Article_IndigenousWomenSAccessToMatern.pdf

📥 Baixando PDF de: https://github.com/biel1205/testenovembro/raw/main/Akter2018_Article_IndigenousWomenSAccessToMatern.pdf
✅ Texto extraído: 51164 caracteres
⚠️  Poucos termos relevantes encontrados: ['SUS']
   O artigo pode não focar em saúde da mulher no SUS
   Continuar mesmo assim? (s/n): s

🧠 Analisando artigo e gerando resposta...
⏳ Isso pode levar 2-5 minutos...
✅ Encontradas 10 seções relevantes
🧠 Consultando DeepSeek Coder V2...

🔍 Verificando consistência da resposta...
🧠 Consultando DeepSeek Coder V2...

🎯 RESPOSTA FINAL
**